# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
import numpy as np

quantity = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
quantity.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
customers = data.pivot_table(index='ProductName',columns='CustomerID', values='Quantity').fillna(0)
customers.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform

matrix = squareform(pdist(customers.T, 'euclidean'))
distances = pd.DataFrame(1/(1+matrix), index = customers.columns, columns = customers.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
specific_ID = 33
similarities = distances[specific_ID].sort_values(ascending= False)
similarities.head(6)

CustomerID
33      1.000000
3909    0.095358
264     0.093953
3531    0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
top_five = list(similarities[1:6].index)
top_five.sort()
top_five_records = quantity.loc[top_five]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
ranked_products = top_five_records.groupby(['ProductName']).agg({'Quantity':'sum'})
ranked_products.sort_values(by='Quantity', ascending=False).head()

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
specific_ID = 33
df = pd.merge(ranked_products,customers[specific_ID], on = 'ProductName')
df['NotBought'] = df.Quantity - df[specific_ID]
df.sort_values(by='NotBought', ascending=False).head(7) # Get top 7

,Quantity,33,NotBought
ProductName,,,
"Salsify, Organic",4,1.0,3.0
Chocolate - Dark,3,0.0,3.0
Wine - Charddonnay Errazuriz,3,0.0,3.0
Bay Leaf,3,0.0,3.0
Wine - Ej Gallo Sierra Valley,3,0.0,3.0
"Oranges - Navel, 72",3,0.0,3.0
Pecan Raisin - Tarts,3,0.0,3.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
recommendations = {}
all_customers = data['CustomerID'].unique()
all_customers.sort()

def suggest_products_for_customer(cust):
    similar_customers = list(distances[cust].sort_values(ascending= False)[1:6].index)
    similar_records = quantity.loc[similar_customers].groupby(['ProductName']).agg({'Quantity':'sum'})
    df = pd.merge(similar_records,customers[cust], on = 'ProductName')
    df['NotBought'] = df.Quantity - df[cust]
    return list(df.sort_values(by='NotBought', ascending=False).index[0:5])

for i in all_customers:
    items = suggest_products_for_customer(i)
    recommendations.update({i:items})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
df = pd.DataFrame(recommendations).T.rename(columns={0:'Item1',1:'Item2',2:'Item3', 3:'Item4',4:'Item5'})
df.head()

,Item1,Item2,Item3,Item4,Item5
33,"Salsify, Organic",Chocolate - Dark,Wine - Charddonnay Errazuriz,Bay Leaf,Wine - Ej Gallo Sierra Valley
200,Muffin - Carrot Individual Wrap,Soup - Campbells Bean Medley,Pork - Kidney,Bay Leaf,Pomello
264,Cheese - Parmesan Grated,Quiche Assorted,Crush - Cream Soda,Soup - Campbells Tomato Ravioli,"Pepper - White, Ground"
356,Butter - Unsalted,Juice - Lime,"Pork - Bacon, Double Smoked",Sage - Ground,Cup - Translucent 7 Oz Clear
412,Wine - Blue Nun Qualitatswein,Table Cloth 81x81 White,Butter - Unsalted,Beef - Top Sirloin,"Pepper - Black, Whole"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
matrix = squareform(pdist(customers.T, 'cosine'))
distances = pd.DataFrame(1/(1+matrix), index = customers.columns, columns = customers.columns)

for i in all_customers:
    items = suggest_products_for_customer(i)
    recommendations.update({i:items})
    
df = pd.DataFrame(recommendations).T.rename(columns={0:'Item1',1:'Item2',2:'Item3', 3:'Item4',4:'Item5'})
df.head()

,Item1,Item2,Item3,Item4,Item5
33,Pasta - Cheese / Spinach Bauletti,Cake - Box Window 10x10x2.5,Durian Fruit,Phyllo Dough,Beef - Ground Medium
200,Blueberries,Olives - Kalamata,Pasta - Orecchiette,Pop Shoppe Cream Soda,Soup - Campbells Tomato Ravioli
264,"Water - Mineral, Natural",Chicken - Wieners,Wine - Sogrape Mateus Rose,Steam Pan - Half Size Deep,"Lemonade - Natural, 591 Ml"
356,Tea - Herbal Sweet Dreams,Chicken - Soup Base,Cream Of Tartar,Ecolab - Lime - A - Way 4/4 L,Cod - Black Whole Fillet
412,Halibut - Fletches,Wine - Vidal Icewine Magnotta,Beans - Wax,Crackers - Trio,Pork - Belly Fresh
